In [36]:
import requests      # Библиотека для отправки запросов
import numpy as np   # Библиотека для матриц, векторов и линала
import pandas as pd  # Библиотека для табличек
import time          # Библиотека для тайм-менеджмента
from bs4 import BeautifulSoup
import re
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from fake_useragent import UserAgent
from tqdm import tqdm
from webdriver_manager.firefox import GeckoDriverManager
from datetime import datetime

In [25]:
MonthDict={ "января" : 1,
      "февраля": 2,
       "марта": 3,
       "апреля": 4,
       "мая": 5,
       "июня": 6,
       "июля": 7,
       "августа": 8,
       "сентября": 9,
       "октября": 10,
       "ноября": 11,
       "декабря": 12
}

In [3]:
driver = webdriver.Firefox(executable_path=GeckoDriverManager().install())

[WDM] - Driver [/home/victor/.wdm/drivers/geckodriver/linux64/v0.27.0/geckodriver] found in cache


In [4]:
#from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

#cap = DesiredCapabilities().FIREFOX
#cap["marionette"] = False

In [5]:
#options = Options()
#options.binary_location = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe"
#driver = webdriver.Chrome(chrome_options=options, executable_path=r"C:\Program Files (x86)\Google\Chrome\chromedriver.exe", )

In [4]:
#обозначили сайт, проверили, чтобы 200 ок
page_link = 'https://www.facebook.com/groups/realtor.russia'
response = requests.get(page_link, headers={'User-Agent': UserAgent().chrome})
response

Error occurred during loading data. Trying to use cache server https://fake-useragent.herokuapp.com/browsers/0.1.11
Traceback (most recent call last):
  File "/home/victor/anaconda3/lib/python3.7/site-packages/fake_useragent/utils.py", line 67, in get
    context=context,
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 222, in urlopen
    return opener.open(url, data, timeout)
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 525, in open
    response = self._open(req, data)
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 543, in _open
    '_open', req)
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 503, in _call_chain
    result = func(*args)
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 1347, in http_open
    return self.do_open(http.client.HTTPConnection, req)
  File "/home/victor/anaconda3/lib/python3.7/urllib/request.py", line 1322, in do_open
    r = h.getresponse()
  Fi

<Response [200]>

In [5]:
driver.get(page_link)

In [17]:
for i in range(25):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

In [18]:
selen_page = driver.page_source
soup = BeautifulSoup(selen_page,'html.parser')

In [19]:
soup

<html class="_9dls" dir="ltr" id="facebook" lang="ru"><head><link data-badged-icon="https://static.xx.fbcdn.net/rsrc.php/ye/r/Ta8_J_nYekI.ico" data-default-icon="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" href="https://static.xx.fbcdn.net/rsrc.php/yD/r/d4ZIVX-5C-b.ico" rel="shortcut icon"/><meta content="width=device-width,initial-scale=1,maximum-scale=2,shrink-to-fit=no" name="viewport"/><meta content="Группы Facebook" name="application-name"/><meta content="https://www.facebook.com/groups/" name="application-url"/><meta content="Риэлторы: информация и аргументы для клиентов: 23 583 участника. Эта группа создана для обмена профессиональной информацией. Особенно приветствуются..." name="description"/><link href="https://www.facebook.com/groups/realtor.russia/" rel="canonical"/><link href="https://m.facebook.com/groups/realtor.russia/?locale2=ru_RU" media="only screen and (max-width: 640px)" rel="alternate"/><link href="https://m.facebook.com/groups/realtor.russia/?local

In [20]:
soup.findAll('div', attrs = {'class':"kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"})
#текст постов

[<div class="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"><div dir="auto" style="text-align: start;">Эта группа создана для обмена профессиональной информацией. Особенно приветствуются здесь ссылки на полезные для риэлтора ресурсы и на информацию, которая может содержать аргументы для работы с клиентами.</div></div>,
 <div class="kvgmc6g5 cxmmr5t8 oygrvhab hcukyx3x c1et5uql ii04i59q"><div dir="auto" style="text-align: start;">Коллеги, а как вы относитесь видимо тоже к коллегам, рекламирующим и предлагающим объект от агенства, а когда приходишь в офис агенства на внесение аванса/задатка оказывается, что предлагают заключить соглашение с собственником, внести средства собственнику, получить расписку от собственника. И ни где нет следа об агенстве. То есть, если собственник откажется проводить сделку, агенство окажется в стороне... деньги трясти с собственника... </div><div dir="auto" style="text-align: start;">схема понятно работающая, но когда п... <div class="oajrlxb2 g5ia77u

In [21]:
b = soup.findAll('span', attrs = {'class':"gpro0wi8 j1lvzwm4 stjgntxs ni8dbmo4 q9uorilb"})

In [22]:
a = soup.findAll('b', attrs = {'class':"b6zbclly myohyog2 l9j0dhe7 aenfhxwr l94mrbxd ihxqhq3m nc684nl6 t5a262vz sdhka5h4"})

In [106]:
parsed_ls = []
for i in range(len(b)):
    parsed = (b[i].text.replace('=','').replace('·','').replace('\xa0', ''))
    parsed_ls.append(parsed)

In [108]:
for i in range(1, len(parsed_ls)):
    parsed_ls[i] = parsed_ls[i].replace(' ', '')
    parsed_ls[i] = parsed_ls[i].replace('г.Москва', '')
    if 'ч.' in parsed_ls[i]:
        parsed_ls[i] = '14сентября2020'
    if 'мин.' in parsed_ls[i]:
        parsed_ls[i] = '14сентября2020'
    if 'Вчера' in parsed_ls[i]:
        parsed_ls[i] = '13сентября2020'

In [109]:
for i in range(1, len(parsed_ls)):    
    if (parsed_ls[i][-3]==':')==True:
        print('suka')
        parsed_ls[i]= parsed_ls[i][:-6]+'2020'
    if parsed_ls[i][:2].isnumeric():
        print(i)
        month = str(MonthDict[parsed_ls[i][2:-4]])
        date = (parsed_ls[i][:2])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year
    elif parsed_ls[16][:2].isnumeric()==False:
        month = str(MonthDict[parsed_ls[i][1:-4]])
        date = (parsed_ls[i][:1])
        year = (parsed_ls[i][-4:])
        parsed_ls[i] = date+'/'+month+'/'+year

1
2
3
4
suka
5
6
suka
7
8
9
10
11
suka
12
13
14
15
suka
suka
17
18
19
20
21
22


In [97]:
parsed_ls[15][:2].isnumeric()

True

In [110]:
parsed_ls

['13 мая 2014 г.',
 '14/9/2020',
 '14/9/2020',
 '14/9/2020',
 '14/9/2020',
 '11/9/2020',
 '14/9/2020',
 '11/9/2020',
 '14/9/2020',
 '14/9/2020',
 '14/9/2020',
 '13/9/2020',
 '10/9/2020',
 '14/9/2020',
 '14/9/2020',
 '13/9/2020',
 '9/9/2020',
 '12/9/2020',
 '14/9/2020',
 '14/9/2020',
 '14/9/2020',
 '13/9/2020',
 '14/9/2020']